In [1]:
import pandas as pd
academy_awards = pd.read_csv('academy_awards.csv', encoding = 'ISO-8859-1')

# Data exploration

In [2]:
academy_awards.head(10)
academy_awards['Unnamed: 8'].value_counts()

 understanding comedy genius - Mack Sennett.""    1
*                                                 1
Name: Unnamed: 8, dtype: int64

# Data cleaning

In [3]:
academy_awards['Year'] = academy_awards['Year'].str[0:4]
academy_awards['Year'] = academy_awards['Year'].astype('int64')
later_than_2000 = academy_awards[academy_awards['Year'] > 2000]
award_categories = ['Actor -- Leading Role', 'Actor -- Supporting Role', 'Actress -- Leading Role', 'Actress -- Supporting Role']
nominations = later_than_2000[later_than_2000['Category'].isin(award_categories)]

In [4]:
replace_dict = {'YES': 1, 'NO': 0}
nominations['Won?'] = nominations['Won?'].map(replace_dict)
nominations['Won'] = nominations['Won?']
to_drop = ['Won?', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10']
final_nominations = nominations.drop(to_drop, axis = 1)

/dataquest/system/env/python3/lib/python3.4/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/dataquest/system/env/python3/lib/python3.4/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [5]:
additional_info_one = final_nominations['Additional Info'].str.rstrip("'}")
additional_info_two = additional_info_one.str.split("{'")
movie_names = additional_info_two.str[0]
characters = additional_info_two.str[1]

In [6]:
final_nominations['Movie'] = movie_names
final_nominations['Character'] = characters
final_nominations.head(10)

Year                  Category          Nominee  \
0  2010     Actor -- Leading Role    Javier Bardem   
1  2010     Actor -- Leading Role     Jeff Bridges   
2  2010     Actor -- Leading Role  Jesse Eisenberg   
3  2010     Actor -- Leading Role      Colin Firth   
4  2010     Actor -- Leading Role     James Franco   
5  2010  Actor -- Supporting Role   Christian Bale   
6  2010  Actor -- Supporting Role      John Hawkes   
7  2010  Actor -- Supporting Role    Jeremy Renner   
8  2010  Actor -- Supporting Role     Mark Ruffalo   
9  2010  Actor -- Supporting Role    Geoffrey Rush   

                          Additional Info  Won                    Movie  \
0                      Biutiful {'Uxbal'}    0                Biutiful    
1           True Grit {'Rooster Cogburn'}    0               True Grit    
2  The Social Network {'Mark Zuckerberg'}    0      The Social Network    
3    The King's Speech {'King George VI'}    1       The King's Speech    
4              127 Hours {'Aron Ralston'}    0               127 Hours    
5            The Fighter {'Dicky Eklund'}    1             The Fighter    
6              Winter's Bone {'Teardrop'}    0           Winter's Bone    
7             The Town {'James Coughlin'}    0                The Town    
8         The Kids Are All Right {'Paul'}    0  The Kids Are All Right    
9      The King's Speech {'Lionel Logue'}    0       The King's Speech    

         Character  
0            Uxbal  
1  Rooster Cogburn  
2  Mark Zuckerberg  
3   King George VI  
4     Aron Ralston  
5     Dicky Eklund  
6         Teardrop  
7   James Coughlin  
8             Paul  
9     Lionel Logue

In [7]:
final_nominations = final_nominations.drop('Additional Info', axis = 1)

In [8]:
import sqlite3
conn = sqlite3.connect('nominations.db')

In [9]:
final_nominations.to_sql('nominations', conn, index = False)

# Verifying SQL

In [13]:
q1 = "pragma table_info(nominations)"
q2 = "SELECT * from nominations LIMIT 10"
print(conn.execute(q1).fetchall())
print(conn.execute(q2).fetchall())
conn.close()

[(0, 'Year', 'INTEGER', 0, None, 0), (1, 'Category', 'TEXT', 0, None, 0), (2, 'Nominee', 'TEXT', 0, None, 0), (3, 'Won', 'INTEGER', 0, None, 0), (4, 'Movie', 'TEXT', 0, None, 0), (5, 'Character', 'TEXT', 0, None, 0)]
[(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful ', 'Uxbal'), (2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit ', 'Rooster Cogburn'), (2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network ', 'Mark Zuckerberg'), (2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech ", 'King George VI'), (2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours ', 'Aron Ralston'), (2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter ', 'Dicky Eklund'), (2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone ", 'Teardrop'), (2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town ', 'James Coughlin'), (2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right ', 'Paul'), (2